In [1]:
# Import packages
import pandas as pd
import numpy as np
import datetime
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, RobustScaler, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, cross_validate, KFold
from sklearn.metrics import accuracy_score, mean_squared_error, cohen_kappa_score, mean_absolute_error, f1_score
from sklearn.utils import shuffle
from imblearn.over_sampling import SMOTE, SMOTENC
import matplotlib.pyplot as plt
from statsmodels.graphics.mosaicplot import mosaic
from sklearn.decomposition import PCA
from scipy import stats
from math import sqrt


import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, GaussianNoise
from keras.optimizers import SGD
from keras import regularizers
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

Using TensorFlow backend.


In [2]:
path = "/Users/Stijn/Documents/Master Data Science and Society/Block 3/thesis/code/thesis_Code/"
mood = pd.read_csv(path+'mood_imputed_median.csv', sep = ',', index_col=0)
mood['anxious_last'] = mood.groupby('user_id')['anxious'].shift()
mood.loc[(pd.isnull(mood.anxious_last)), 'anxious_last'] = mood['anxious']
mood_anxious = mood.drop(["bored", "content", "cheerful", "user_id", "response_time"], axis=1)

In [3]:
# Normalize predictors
mood_normz = mood_anxious.copy()
features_to_normalize = ['day_time_window', 'average_TimeUse', 'messaging', 'socialnetworking', 'otherapp']
mood_normz[features_to_normalize] = mood_normz[features_to_normalize].apply(lambda x:(x-x.min()) / (x.max()-x.min()))

In [4]:
# X = predictors, y = target
X = mood_normz.iloc[:,1:]
y = mood_normz.iloc[:, 0:1]

# Convert float to int
y = y.astype(int)

In [5]:
X, y = shuffle(X, y)

In [6]:
X = X.iloc[:,:-1]
X_onlylastmood = X.iloc[:,-1:]

In [8]:
y['anxious'].value_counts()
len(y)

13221

In [9]:
def macroaverage_mae(test, pred):
    mae_0, mae_1, mae_2, mae_3, mae_4, mae_5 = (0,0,0,0,0,0)

    for i in range(len(pred)):
        if test[i] == 0 and pred[i] != 0:
            mae_0 += (abs(test[i]-pred[i]))
        if test[i] == 1 and pred[i] != 1:
            mae_1 += (abs(test[i]-pred[i]))
        if test[i] == 2 and pred[i] != 2:
            mae_2 += (abs(test[i]-pred[i]))
        if test[i] == 3 and pred[i] != 3:
            mae_3 += (abs(test[i]-pred[i]))
        if test[i] == 4 and pred[i] != 4:
            mae_4 += (abs(test[i]-pred[i]))
        if test[i] == 5 and pred[i] != 5:
            mae_5 += (abs(test[i]-pred[i]))

    cnt_0, cnt_1, cnt_2, cnt_3, cnt_4, cnt_5 = (0,0,0,0,0,0)
    for z in range(len(test)):
        if test[z] == 0:
            cnt_0 += 1
        if test[z] == 1:
            cnt_1 += 1
        if test[z] == 2:
            cnt_2 += 1
        if test[z] == 3:
            cnt_3 += 1
        if test[z] == 4:
            cnt_4 += 1
        if test[z] == 5:
            cnt_5 += 1

    mae_macroaverage = ((mae_0/cnt_0) + (mae_1/cnt_1) + (mae_2/cnt_2) + (mae_3/cnt_3) + (mae_4/cnt_4) + (mae_5/cnt_5)) / 6
    return mae_macroaverage 

In [10]:
print((7206/13221) * 100)
print((2548/13221) * 100)
print((1946/13221) * 100)
print((1012/13221) * 100)
print((384/13221) * 100)
print((125/13221) * 100)


54.504197867029724
19.27236971484759
14.719007639361623
7.654489070418275
2.904470161107329
0.9454655472354587


In [11]:
n_split=10
f1_scores = []
mmae_scores = []
f1_scores_classes = []
for train_index, test_index in KFold(n_splits = n_split, random_state=2, shuffle=True).split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    pred = []
    for a in range(len(y_test)):
        pred.append(np.random.choice(np.arange(0, 6), p=[0.5450, 0.1927, 0.1473, 0.0765, 0.0290, 0.0095]))
    test = y_test.values
    
    f1_scores_classes.append(f1_score(test, pred, average=None))
    f1_scores.append(round(f1_score(test, pred, average='weighted'), 4))
    mmae_scores.append(macroaverage_mae(test, pred))
    

In [12]:
print("Macro-averaged F1-scores: ", f1_scores)
print("Mean MA F1-scores: ", round(np.mean(f1_scores), 4))
print("Std MA F1-scores: ", round(np.std(f1_scores), 4))

Macro-averaged F1-scores:  [0.3604, 0.3648, 0.3455, 0.3787, 0.3655, 0.3552, 0.35, 0.3747, 0.373, 0.3628]
Mean MA F1-scores:  0.3631
Std MA F1-scores:  0.0102


In [13]:
mmae_scores_rounded = []
for a in mmae_scores:
    for b in a:
        mmae_scores_rounded.append(round(b, 4))
        
print("Macro-averaged MAE: ", mmae_scores_rounded)
print("Mean MMAE-scores: ", round(np.mean(mmae_scores), 4))
print("Std MMAE-scores: ", round(np.std(mmae_scores), 4))

Macro-averaged MAE:  [2.1291, 2.2072, 2.174, 2.0953, 2.1666, 2.0829, 2.1798, 2.1017, 2.073, 2.1808]
Mean MMAE-scores:  2.139
Std MMAE-scores:  0.0458


In [14]:
print("Mean F1-score per class: ", [round(np.mean(x), 4) for x in zip(*f1_scores_classes)])
print("STD F1-score per class: ", [round(np.std(x), 4) for x in zip(*f1_scores_classes)])

Mean F1-score per class:  [0.5459, 0.1991, 0.1361, 0.0786, 0.0285, 0.008]
STD F1-score per class:  [0.0134, 0.0253, 0.0179, 0.0386, 0.0165, 0.024]
